In [2]:
import requests
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from bs4 import BeautifulSoup
import re
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch
from transformers import pipeline
from config import config
from pyBM25 import BM25

In [51]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
classifier = pipeline(task='sentiment-analysis',model=model,tokenizer=tokenizer)
max_tokens = int(tokenizer.model_max_length)

In [4]:
stopwords=[]
with open('stopwords.txt') as f:
    stopwords.append(f.read().splitlines())

In [6]:
bm25 = BM25(stopwords=stopwords)

In [64]:
url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/search/NewsSearchAPI"

q = "PLUG"
d_from ="11/01/2021"
d_to = "null"

querystring = {"q":q,"pageNumber":"1","pageSize":"150","autoCorrect":"false","fromPublishedDate":d_from,"toPublishedDate":d_to}

headers = {
    'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
    'x-rapidapi-key': "954c6fbd55msh55107080293cea6p148e40jsn1db2b31b95fd"
    }

response = requests.request("GET", url, headers=headers, params=querystring)


In [65]:
response = response.json()

In [66]:
for i in response['value'][0]:
    print(i)

id
title
url
description
body
snippet
keywords
language
isSafe
datePublished
provider
image


In [67]:
corpus=[]
for i in response['value']:
    corpus.append(i['body'])

In [68]:
print(response['value'][0]['title'])

Why Ballard Power Bloom Energy and Nikola Stocks All Dropped Today -- But Plug Didn't


In [69]:
for i in response['value']:
    print(i['url'])

https://www.fool.com/investing/2021/11/09/ballard-power-bloom-energy-nikola-stocks-drop/?source=iedfolrf0000001
https://timesofmalta.com/articles/view/shopify-plug-in-introduced-to-ecabs-fastdrop-platform.913614
https://arstechnica.com/?p=1811536
https://nesn.com/2021/11/tom-brady-responds-to-aaron-rodgers-question-with-shameless-plug/
https://www.refinery29.com/en-us/2021/11/10740548/maude-cone-butt-plug?utm_source=feed&utm_medium=rss
https://in.pcmag.com/smart-plugs/146046/kasa-smart-wi-fi-outdoor-plug-in-dimmer-kp-405
https://insideevs.com/news/546198/uk-plugin-car-sales-october2021/
https://trailer-wiring-diagram.com/7-core-trailer-wiring-diagram/narva-trailer-plug-wiring-diagram-7-core-trailer-wiring-diagram/
https://investorplace.com/2021/11/plug-powers-rally-is-just-beginning/
https://www.prweb.com/releases/autonomous_holds_7_day_launch_event_with_savings_up_to_6k_for_plug_and_play_building/prweb18317826.htm
http://zacks.com/stock/research/plug/brokerage-reports?searchby=ticker&

In [70]:
q = "PLUG"
bm25 = BM25()
bm25.fit(corpus)
result_vec = bm25.transform(q, corpus)
print(result_vec)

[0.29185733 0.30904349 0.43654851 0.27129046 0.34126776 0.4615587
 0.44669564 0.         0.46142567 0.154225   0.         0.
 0.14630514 0.         0.3196947  0.36464667 0.4305363  0.
 0.31081692 0.159021   0.40221191 0.33664041 0.34165752 0.31142481
 0.40628177 0.37634468 0.31851863 0.         0.45862815 0.45920416
 0.3951347  0.20223834 0.36562126 0.40979027 0.38542192 0.43247627
 0.30043076 0.         0.21721695 0.         0.         0.15677092
 0.22156069 0.27645681 0.37265274 0.41062643 0.43978796 0.36206374
 0.36522505 0.29027189]


In [132]:
# from requests_html import AsyncHTMLSession
# session = AsyncHTMLSession()
# r = session.get(uurl)
# r = r.result()
# print(r.html.text)

In [71]:
def build_corpus_fromurl(url_list):
    corpus = []
    headers = {"User-Agent":"Mozilla/5.0"}
    for i in url_list:
        response = requests.get(url=i['url'],headers=headers)
        if response.status_code==200:
            soup = BeautifulSoup(response.content, 'html.parser')
            corpus.append(soup.get_text())
    return corpus

In [14]:
#corpus = build_corpus_fromurl(response['value'])

In [72]:
def get_pgraphs(soup):
    pgraphs=[]
    if soup.find('p')!=None:
        freshsoup = BeautifulSoup(str(soup("p")),'html.parser')
        for p in freshsoup("p"):
            pgraphs.append(p.get_text())
    else:
        freshsoup = re.split('\n\n',soup.get_text())
        for x in range(0,len(freshsoup)):
            if len(freshsoup[x])>3:
                pgraphs.append(freshsoup[x])
    return pgraphs

In [80]:
def get_subdocs(pgraphs, max_tokens, min_sen=4):
    sub_docs=[]

    for x in range(0, len(pgraphs)):
        sen_cnt = len(re.split('\n|\. ',pgraphs[x]))
        tkns = int(len(tokenizer(pgraphs[x])['input_ids']))
        
        if tkns>=max_tokens:
                cut_point = pgraphs[x].rfind(".",0,int(len(pgraphs[x])/2))+1
                sen2 = pgraphs[x][cut_point:]
                sen = pgraphs[x][0:cut_point]
                sub_docs.append(sen)
                sub_docs.append(sen2)
        else:
            sub_docs.append(pgraphs[x])
        

    
    return sub_docs

In [81]:
doc_cutoff = 0.25
subdoc_cutoff = 0

for i in response['value']:
    print(i['url'])
    headers = {"User-Agent":"Mozilla/5.0"}
    uspon = requests.get(url=i['url'],headers=headers)
    if uspon.status_code==200:
        soup = BeautifulSoup(uspon.content, 'html.parser')

        pgraphs = get_pgraphs(soup)
        sub_docs = get_subdocs(pgraphs,max_tokens)
        
        bm25.fit(sub_docs)
        result_vec = bm25.transform(q, sub_docs)
        
        sentiments = []
        for x in range(0,len(sub_docs)):
            if result_vec[x]>0:
                print(len(tokenizer(sub_docs[x])['input_ids']))
                s = classifier(sub_docs[x])
                scr = s[0]['score']
                if s[0]['label']=="NEGATIVE":
                    scr=scr*-1
                sentiments.append(scr)
    else:
        print(uspon.status_code)

https://www.fool.com/investing/2021/11/09/ballard-power-bloom-energy-nikola-stocks-drop/?source=iedfolrf0000001
96
85
65
https://timesofmalta.com/articles/view/shopify-plug-in-introduced-to-ecabs-fastdrop-platform.913614
78
67
https://arstechnica.com/?p=1811536
56
73
73
37
https://nesn.com/2021/11/tom-brady-responds-to-aaron-rodgers-question-with-shameless-plug/
24
https://www.refinery29.com/en-us/2021/11/10740548/maude-cone-butt-plug?utm_source=feed&utm_medium=rss
11
377
384
30
https://in.pcmag.com/smart-plugs/146046/kasa-smart-wi-fi-outdoor-plug-in-dimmer-kp-405
165
138
104
183
83
135
102
153
https://insideevs.com/news/546198/uk-plugin-car-sales-october2021/
53
29
66
35
27
https://trailer-wiring-diagram.com/7-core-trailer-wiring-diagram/narva-trailer-plug-wiring-diagram-7-core-trailer-wiring-diagram/
https://investorplace.com/2021/11/plug-powers-rally-is-just-beginning/
12
79
91
99
54
109
80
62
125
20
37
https://www.prweb.com/releases/autonomous_holds_7_day_launch_event_with_savings_

In [82]:
print(sentiments)

[-0.981147289276123, -0.9988498687744141, 0.9922612309455872, 0.9609275460243225]


In [184]:
q = "PLUG"
bm25 = BM25()
bm25.fit(corpus)
result_vec = bm25.transform(q, subs)
print(result_vec)
print("Length: ",len(result_vec))
print("Mean Score: ",np.mean(result_vec))
print("Median Score: ", np.median(result_vec))

[0.0433606  0.04105189 0.03070828 0.        ]
Length:  4
Mean Score:  0.028780191640045347
Median Score:  0.03588008440179806
